In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from sklearn.model_selection import train_test_split

In [2]:
# Load dataset
df = pd.read_csv('farms_dataset.csv')
df['label'] = df['Farm Type'].apply(lambda x: 0 if x == 'Random' else 1)

# Custom Dataset Class
class FarmDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 4]
        # Check if the root directory is already in the image path
        if not img_path.startswith(self.root_dir):
            img_path = os.path.join(self.root_dir, img_path)
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx, -1]

        if self.transform:
            image = self.transform(image)

        return image, label


# Data Transforms
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
])

# CNN Model Definition
class FarmNet(nn.Module):
    def __init__(self):
        super(FarmNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 50 * 50, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 50 * 50)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [21]:
model = FarmNet()

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Split data into train, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.15/0.85, random_state=42)

# Create datasets
train_dataset = FarmDataset(train_df, './data_satelite/', transform=transform)
val_dataset = FarmDataset(val_df, './data_satelite/', transform=transform)
test_dataset = FarmDataset(test_df, './data_satelite/', transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to calculate accuracy
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Training Loop
for epoch in range(10):  # number of epochsa
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_accuracy = calculate_accuracy(val_loader, model)
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {val_accuracy}%')

print('Finished Training')

# Optionally, evaluate on test set
test_accuracy = calculate_accuracy(test_loader, model)
print(f'Test Accuracy: {test_accuracy}%')

Epoch 1, Loss: 1.640096108118693, Validation Accuracy: 65.0%
Epoch 2, Loss: 0.5235778788725535, Validation Accuracy: 87.5%
Epoch 3, Loss: 0.45914871990680695, Validation Accuracy: 87.5%
Epoch 4, Loss: 0.41278985639413196, Validation Accuracy: 80.0%
Epoch 5, Loss: 0.3375050375858943, Validation Accuracy: 90.0%
Epoch 6, Loss: 0.250460813442866, Validation Accuracy: 90.0%
Epoch 7, Loss: 0.17000082383553186, Validation Accuracy: 85.0%
Epoch 8, Loss: 0.13157116870085397, Validation Accuracy: 92.5%
Epoch 9, Loss: 0.07266261801123619, Validation Accuracy: 95.0%
Epoch 10, Loss: 0.049474628642201424, Validation Accuracy: 90.0%
Finished Training
Test Accuracy: 95.0%


In [3]:
model = FarmNet()  # Replace with your model and architecture
model.load_state_dict(torch.load('farmnet_model/farmnet_model.pth'))

<All keys matched successfully>

In [5]:
import wandb
import random

# Start a new W&B run
with wandb.init(project="farmnet") as run:
    # Simulate logging model metricsWW
    run.log({"acc": test_accuracy})

    # Save the dummy model to W&B
    best_model = wandb.Artifact("farmnet_model_1", type="model")
    best_model.add_file("farmnet_model/farmnet_model.pth")
    run.log_artifact(best_model)

    # Link the model to the Model Registry
    run.link_artifact(best_model, "model-registry/FarmNet")

    run.finish()

wandb: Currently logged in as: filipz. Use `wandb login --relogin` to force relogin


wandb: Waiting for artifact farmnet_model_1 to be committed...

wandb: Committed artifact filipz/farmnet/farmnet_model_1:v0


acc,▁
acc,0.95


In [8]:
run = wandb.init(project="farmnet", job_type='inference')
artifact = run.use_artifact("farmnet_model_1:latest", type='model')
artifact_dir = artifact.download()
wandb.finish()

wandb: Downloading large artifact farmnet_model_1:latest, 312.60MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.5


#### Tesing Finetunig Resnet (worse result)

In [11]:
import torchvision.models as models

# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # Assuming binary classification

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/home/filip/anaconda3/envs/py3_10_torch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/filip/anaconda3/envs/py3_10_torch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/filip/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:06<00:00, 7.12MB/s]


In [13]:
# Split data into train, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.15/0.85, random_state=42)

# Create datasets
train_dataset = FarmDataset(train_df, './data_satelite/', transform=transform)
val_dataset = FarmDataset(val_df, './data_satelite/', transform=transform)
test_dataset = FarmDataset(test_df, './data_satelite/', transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to calculate accuracy
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Training Loop
for epoch in range(10):  # number of epochsa
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_accuracy = calculate_accuracy(val_loader, model)
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {val_accuracy}%')

print('Finished Training')

# Optionally, evaluate on test set
test_accuracy = calculate_accuracy(test_loader, model)
print(f'Test Accuracy: {test_accuracy}%')

Epoch 1, Loss: 2.0425245066483817, Validation Accuracy: 87.5%
Epoch 2, Loss: 0.5600331525007883, Validation Accuracy: 87.5%
Epoch 3, Loss: 0.49221118291219074, Validation Accuracy: 87.5%
Epoch 4, Loss: 0.4861273020505905, Validation Accuracy: 87.5%
Epoch 5, Loss: 0.4201107670863469, Validation Accuracy: 87.5%
Epoch 6, Loss: 0.5022070606549581, Validation Accuracy: 95.0%
Epoch 7, Loss: 0.39630846679210663, Validation Accuracy: 80.0%
Epoch 8, Loss: 0.3230523963769277, Validation Accuracy: 90.0%
Epoch 9, Loss: 0.3381749428808689, Validation Accuracy: 92.5%
Epoch 10, Loss: 0.5580974308152994, Validation Accuracy: 67.5%
Finished Training
Test Accuracy: 77.5%
